In [2]:
# Install necessary libraries
!pip install pandas scikit-learn xgboost streamlit

import pandas as pd

# URLs of the datasets
train_url = "https://media.githubusercontent.com/media/rishi-wqd190004/dash_app_challenge/main/datasets/fraudTrain.csv"
test_url = "https://github.com/rishi-wqd190004/dash_app_challenge/blob/main/datasets/fraudTest.csv?raw=true"

# Load datasets
train_df = pd.read_csv(train_url)
test_df = pd.read_csv(test_url)

# Save datasets locally (optional)
train_df.to_csv('fraudTrain.csv', index=False)
test_df.to_csv('fraudTest.csv', index=False)

# Display the first few rows of the training dataset
train_df.head()


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [9]:
import pandas as pd
from datetime import datetime
import math

# Define the updated df_cleaner function
def df_cleaner(df: pd.DataFrame, name: str):
    df['dob'] = pd.to_datetime(df['dob'])
    df['age_in_days'] = (datetime.now() - df['dob']).dt.days
    df['age'] = df['age_in_days'] / 365
    df['age'] = df['age'].apply(math.floor)
    df['age'] = df['age'].astype(int)

    df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
    df['hour'] = df['trans_date_trans_time'].dt.hour

    df.drop(columns=['Unnamed: 0', 'cc_num', 'merchant', 'first', 'last', 'street', 'city', 'zip',
                     'job', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'dob',
                     'age_in_days', 'trans_date_trans_time'], inplace=True)
    df.rename(columns={'amt': 'amount', 'city_pop': 'city_population'}, inplace=True)
    df.to_csv(f'{name}.csv', index=False)
    return df

# URLs of the datasets
train_url = "https://media.githubusercontent.com/media/rishi-wqd190004/dash_app_challenge/main/datasets/fraudTrain.csv"
test_url = "https://github.com/rishi-wqd190004/dash_app_challenge/blob/main/datasets/fraudTest.csv?raw=true"

# Load datasets
train_df = pd.read_csv(train_url)
test_df = pd.read_csv(test_url)

# Apply cleaning function
train_df = df_cleaner(train_df, 'Train_df')
test_df = df_cleaner(test_df, 'Test_df')

# Concatenate train and test datasets
df = pd.concat([train_df, test_df], axis=0)

# Save cleaned dataset
df.to_csv('cleaned_data_with_lat_long.csv', index=False)

# Display the first few rows of the cleaned dataset
df.head()


,category,amount,gender,state,lat,long,city_population,is_fraud,age,hour
0,misc_net,4.97,F,NC,36.0788,-81.1781,3495,0,36,0
1,grocery_pos,107.23,F,WA,48.8878,-118.2105,149,0,46,0
2,entertainment,220.11,M,ID,42.1808,-112.2620,4154,0,62,0
3,gas_transport,45.00,M,MT,46.2306,-112.1138,1939,0,57,0
4,misc_pos,41.96,M,VA,38.4207,-79.4629,99,0,38,0


In [10]:
from sklearn.preprocessing import OrdinalEncoder
import random

# Load cleaned dataset
df = pd.read_csv('cleaned_data.csv')

# Encoding categorical variables
encoder = OrdinalEncoder()
for col in ['category', 'gender', 'state', 'hour']:
    df[col] = encoder.fit_transform(df[col].astype('category').values.reshape(-1, 1))

# Handle imbalanced dataset
df_fraud = df[df['is_fraud'] == 1]
df_nofraud = df[df['is_fraud'] == 0].sample(n=len(df_fraud), random_state=0)
df_balanced = pd.concat([df_fraud, df_nofraud], axis=0)

# Save balanced dataset
df_balanced.to_csv('balanced_data.csv', index=False)

# Display the distribution of the target variable
df_balanced['is_fraud'].value_counts()


is_fraud
1    9651
0    9651
Name: count, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import joblib

# Load balanced dataset
df_balanced = pd.read_csv('balanced_data.csv')

X = df_balanced.drop('is_fraud', axis=1)
y = df_balanced['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train various models
clfs = [LogisticRegression(), Perceptron(), SVC(), AdaBoostClassifier(), GradientBoostingClassifier(), XGBClassifier(), RandomForestClassifier()]
names = ['Logistic Regression', 'Perceptron', 'SVC', 'Ada Boost Classifier', 'Gradient Boosting Classifier', 'XGB Classifier', 'Random Forest Classifier']

ranking_dic = {}
modelos_dic = {}

for clf, name in zip(clfs, names):
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    ranking_dic[name] = accuracy
    modelos_dic[name] = {'conf_matrix': confusion_matrix(y_test, y_pred), 'accuracy': accuracy}

# Print model accuracies
for name, accuracy in ranking_dic.items():
    print(f'{name} accuracy: {accuracy}')
    print(f'Confusion Matrix:\n {modelos_dic[name]["conf_matrix"]}')

# Save the best model
best_model = GradientBoostingClassifier()
best_model.fit(X_train_scaled, y_train)
joblib.dump(best_model, 'best_model.pkl')


C:\Users\DS\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Logistic Regression accuracy: 0.8464128464128464
Confusion Matrix:
 [[1834   97]
 [ 496 1434]]
Perceptron accuracy: 0.7531727531727531
Confusion Matrix:
 [[1427  504]
 [ 449 1481]]
SVC accuracy: 0.8834498834498834
Confusion Matrix:
 [[1766  165]
 [ 285 1645]]
Ada Boost Classifier accuracy: 0.9303289303289304
Confusion Matrix:
 [[1823  108]
 [ 161 1769]]
Gradient Boosting Classifier accuracy: 0.9624449624449625
Confusion Matrix:
 [[1873   58]
 [  87 1843]]
XGB Classifier accuracy: 0.9774669774669774
Confusion Matrix:
 [[1891   40]
 [  47 1883]]
Random Forest Classifier accuracy: 0.9694379694379694
Confusion Matrix:
 [[1883   48]
 [  70 1860]]


['best_model.pkl']

In [13]:
import joblib

# Assuming `best_model` is your trained model
joblib.dump(best_model, 'best_model.pkl')

['best_model.pkl']